In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import ipywidgets as pyw
import hyperspy.api as hys
import tkinter.filedialog as tkf
from tabulate import tabulate
from sklearn.decomposition import NMF
import malspy
plt.rcParams['font.family'] = 'Times New Roman'

In [ ]:
# create a customized colorbar
color_rep = ["black", "red", "green", "blue", "orange", "purple", "yellow", "lime", 
             "cyan", "magenta", "lightgray", "peru", "springgreen", "deepskyblue", 
             "hotpink", "darkgray"]
print(len(color_rep))
custom_cmap = mcolors.ListedColormap(color_rep)
bounds = np.arange(-1, len(color_rep))
norm = mcolors.BoundaryNorm(boundaries=bounds, ncolors=len(color_rep))
sm = cm.ScalarMappable(cmap=custom_cmap, norm=norm)
sm.set_array([])

cm_rep = ["gray", "Reds", "Greens", "Blues", "Oranges", "Purples"]
print(len(cm_rep))

In [ ]:
def zero_one_rescale(spectrum):
    """
    normalize one spectrum from 0.0 to 1.0
    """
    spectrum = spectrum.clip(min=0.0)
    min_val = np.min(spectrum)
    
    rescaled = spectrum - min_val
    
    if np.max(rescaled) != 0:
        rescaled = rescaled / np.max(rescaled)
    
    return rescaled

In [ ]:
def data_load(adr, rescale=False, crop=None):
    """
    load a spectrum image
    """
    storage = []
    shape = []
    for i, adr in enumerate(adr):
        temp = hys.load(adr)
        if rescale:
            temp.data = temp.data / np.max(temp.data)
        temp = temp.isig[crop[0]:crop[1]]
        temp = temp.data
        print(np.max(temp), np.min(temp))
        print(temp.shape)

        shape.append(temp.shape)
        storage.append(temp)
    
    shape = np.asarray(shape)
    return storage, shape

In [ ]:
def binning_SI(si, bin_y, bin_x, str_y, str_x, offset, depth, rescale=True):
    """
    re-bin a spectrum image
    """
    si = np.asarray(si)
    rows = range(0, si.shape[0]-bin_y+1, str_y)
    cols = range(0, si.shape[1]-bin_x+1, str_x)
    new_shape = (len(rows), len(cols))
    
    binned = []
    for i in rows:
        for j in cols:
            temp_sum = np.mean(si[i:i+bin_y, j:j+bin_x, offset:(offset+depth)], axis=(0, 1))
            if rescale:
                binned.append(zero_one_rescale(temp_sum))
            else:
                binned.append(temp_sum)
            
    binned = np.asarray(binned).reshape(new_shape[0], new_shape[1], depth)
    
    return binned

In [ ]:
def reshape_coeff(coeffs, new_shape):
    """
    reshape a coefficient matrix to restore the original scanning shapes.
    """
    coeff_reshape = []
    for i in range(len(new_shape)):
        temp = coeffs[:int(new_shape[i, 0]*new_shape[i, 1]), :]
        coeffs = np.delete(coeffs, range(int(new_shape[i, 0]*new_shape[i, 1])), axis=0)
        temp = np.reshape(temp, (new_shape[i, 0], new_shape[i, 1], -1))
        #print(temp.shape)
        coeff_reshape.append(temp)
        
    return coeff_reshape

In [ ]:
file_adr = []

In [ ]:
file_adr.extend(tkf.askopenfilenames())
print(len(file_adr))
print(*file_adr, sep="\n")

In [ ]:
num_img = len(file_adr)
print(num_img)

In [ ]:
# load spectrum images
cr_range = [0.1, 0.3, 0.005] # actual input
data_storage, data_shape = data_load(file_adr, rescale=False, crop=cr_range, roll_axis=False)
print(len(data_storage))
print(data_shape)

e_range = np.arange(cr_range[0], cr_range[1], cr_range[2])
print(len(e_range))

In [ ]:
# re-bin spectrum images in order to mitigate noises
bin_y = 1 # binning size (height)
bin_x = 1 # binning size (width)
str_y = 1 # stride width-direction
str_x = 1 # stride height-direction

# actual input
dataset = []
data_shape_new = []
offset = 0
depth = len(e_range)
for img in data_storage:
    print(img.shape)
    processed = binning_SI(img, bin_y, bin_x, str_y, str_x, offset, depth, rescale=True) # include the step for re-scaling the actual input
    print(processed.shape)
    data_shape_new.append(processed.shape)
    dataset.append(processed)
    
data_shape_new = np.asarray(data_shape_new)
print(data_shape_new)

In [ ]:
# create the input dataset
dataset_flat = []
for i in range(num_img):
    dataset_flat.extend(dataset[i].clip(min=0.0).reshape(-1, depth).tolist())
    
dataset_flat = np.asarray(dataset_flat)
print(dataset_flat.shape)

In [ ]:
total_num = len(dataset_flat)
ri = np.random.choice(total_num, total_num, replace=False)

dataset_input = dataset_flat[ri]

In [ ]:
num_comp = 3

In [ ]:
# https://github.com/MotokiShiga/malspy
# NMF with automatic relevance determination and soft orthogonality penalty
model_nmf_ardso = malspy.NMF_ARD_SO(n_components=num_comp, wo=0.1, reps=5, max_itr=100)
model_nmf_ardso.fit(dataset_input)
model_nmf_ardso.plot_spectra(figsize=(6, 3), normalize=False)
ardso_coeffs = model_nmf_ardso.C_
print(ardso_coeffs.shape)
ardso_comp_vectors = model_nmf_ardso.S_
ardso_comp_vectors = np.rollaxis(ardso_comp_vectors, 1, 0)
print(ardso_comp_vectors.shape)

In [ ]:
# convert the coefficient matrix into coefficient maps
coeffs = np.zeros_like(ardso_coeffs)
coeffs[ri] = ardso_coeffs.copy()
coeffs_reshape = reshape_coeff(coeffs, data_shape_new)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5)) # all loading vectors

for i in range(len(ardso_comp_vectors)):
    ax.plot(e_range, ardso_comp_vectors[i], "-", c=color_rep[i+1], label="loading vector %d"%(i+1))
ax.grid()
ax.legend(fontsize="large")
ax.set_xlabel("eV", fontsize=10)
ax.tick_params(axis="both", labelsize=15)
ax.axes.get_yaxis().set_visible(False)

fig.tight_layout()
plt.show()

In [ ]:
# visualize the coefficient maps
if num_img != 1:
    for i in range(len(ardso_comp_vectors)):
        fig, ax = plt.subplots(1, num_img, figsize=(5*num_img, 5))
        for j in range(num_img):
            tmp = ax[j].imshow(coeffs_reshape[j][:, :, i], cmap="viridis")
            ax[j].set_title("loading vector %d map"%(i+1), fontsize=10)
            ax[j].axis("off")
            fig.colorbar(tmp, cax=fig.add_axes([0.92, 0.15, 0.04, 0.7]))
        plt.show()
else:            
    for i in range(len(ardso_comp_vectors)):
        fig, ax = plt.subplots(1, 1, figsize=(5*num_img, 5))
        tmp = ax.imshow(coeffs_reshape[0][:, :, i], cmap="viridis")
        ax.set_title("loading vector %d map"%(i+1), fontsize=10)
        ax.axis("off")
        fig.colorbar(tmp, cax=fig.add_axes([0.92, 0.15, 0.04, 0.7]))
        plt.show()

In [ ]:
import tifffile
for i in range(num_comp):
    tilt_series = []
    for j in range(num_img):
        tilt_series.append(coeffs_reshape[j][:, :, i].astype(np.float32))
    tilt_series = np.asarray(tilt_series)
    tifffile.imsave("tilt_series_latent_%02d.tif"%(i+1), tilt_series)

In [ ]:
# https://github.com/MotokiShiga/malspy
# NMF with soft orthogonality penalty
model_nmf_so = malspy.NMF_SO(n_components=num_comp, wo=0.10, reps=5, max_itr=100)
model_nmf_so.fit(dataset_input)
so_coeffs = model_nmf_so.C_
print(so_coeffs.shape)
so_comp_vectors = model_nmf_so.S_
so_comp_vectors = np.rollaxis(so_comp_vectors, 1, 0)
print(so_comp_vectors.shape)

In [ ]:
# convert the coefficient matrix into coefficient maps
coeffs = np.zeros_like(so_coeffs)
coeffs[ri] = so_coeffs.copy()
coeffs_reshape = reshape_coeff(coeffs, data_shape_new)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5)) # all loading vectors
for i in range(num_comp):
    ax.plot(e_range, so_comp_vectors[i], "-", c=color_rep[i+1], label="loading vector %d"%(i+1))
ax.grid()
ax.legend(fontsize="large")
ax.set_xlabel("eV", fontsize=10)
ax.tick_params(axis="x", labelsize=10)
ax.axes.get_yaxis().set_visible(False)

fig.tight_layout()
plt.show()

In [ ]:
# visualize the coefficient maps
if num_img != 1:
    for i in range(num_comp):
        fig, ax = plt.subplots(1, num_img, figsize=(5*num_img, 5))
        for j in range(num_img):
            tmp = ax[j].imshow(coeffs_reshape[j][:, :, i], cmap="viridis")
            ax[j].set_title("loading vector %d map"%(i+1), fontsize=10)
            ax[j].axis("off")
            fig.colorbar(tmp, cax=fig.add_axes([0.92, 0.15, 0.04, 0.7]))
        plt.show()
else:            
    for i in range(num_comp):
        fig, ax = plt.subplots(1, 1, figsize=(5*num_img, 5))
        tmp = ax.imshow(coeffs_reshape[0][:, :, i], cmap="viridis")
        ax.set_title("loading vector %d map"%(i+1), fontsize=10)
        ax.axis("off")
        fig.colorbar(tmp, cax=fig.add_axes([0.92, 0.15, 0.04, 0.7]))
        plt.show()

In [ ]:
import tifffile
for i in range(num_comp):
    tilt_series = []
    for j in range(num_img):
        tilt_series.append(coeffs_reshape[j][:, :, i].astype(np.float32))
    tilt_series = np.asarray(tilt_series)
    tifffile.imsave("tilt_series_latent_%02d.tif"%(i+1), tilt_series)

In [ ]:
# NMF decomposition (linear dimensionality reduction)
# please visit the below link for detailed information on NMF
# https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html?highlight=nmf#sklearn.decomposition.NMF

skl_nmf = NMF(n_components=num_comp, init="nndsvda", solver="mu", max_iter=2000, 
              verbose=True, beta_loss="frobenius", l1_ratio=0.0, alpha=0.0)

skl_coeffs = skl_nmf.fit_transform(dataset_input)
skl_comp_vectors = skl_nmf.components_
print(skl_coeffs.shape)
print(skl_comp_vectors.shape)

In [ ]:
# convert the coefficient matrix into coefficient maps
coeffs = np.zeros_like(skl_coeffs)
coeffs[ri] = skl_coeffs.copy()
coeffs_reshape = reshape_coeff(coeffs, data_shape_new)

In [ ]:
# visualize loading vectors

fig, ax = plt.subplots(1, 2, figsize=(10, 4)) # all loading vectors
for i in range(num_comp):
    ax[0].plot(e_range, skl_comp_vectors[i], "-", c=color_rep[i+1], label="loading vector %d"%(i+1))
ax[0].grid()
ax[0].legend(fontsize="large")
ax[0].set_xlabel("eV", fontsize=10)
ax[0].tick_params(axis="x", labelsize=10)
ax[0].axes.get_yaxis().set_visible(False)

sel_nmf_comp = [2, 3] # choose several loading vectors to visualize
for i in sel_nmf_comp:
    ax[1].plot(e_range, skl_comp_vectors[i-1], "-", c=color_rep[i], label="loading vector %d"%(i))
ax[1].grid()
ax[1].legend(fontsize="large")
ax[1].set_xlabel("eV", fontsize=10)
ax[1].tick_params(axis="x", labelsize=10)
ax[1].axes.get_yaxis().set_visible(False)

fig.tight_layout()
plt.show()

In [ ]:
# visualize the coefficient maps
if num_img != 1:
    for i in range(num_comp):
        fig, ax = plt.subplots(1, num_img, figsize=(5*num_img, 5))
        for j in range(num_img):
            tmp = ax[j].imshow(coeffs_reshape[j][:, :, i], cmap="viridis")
            ax[j].set_title("loading vector %d map"%(i+1), fontsize=10)
            ax[j].axis("off")
            fig.colorbar(tmp, cax=fig.add_axes([0.92, 0.15, 0.04, 0.7]))
        plt.show()
else:            
    for i in range(num_comp):
        fig, ax = plt.subplots(1, 1, figsize=(5*num_img, 5))
        tmp = ax.imshow(coeffs_reshape[0][:, :, i], cmap="viridis")
        ax.set_title("loading vector %d map"%(i+1), fontsize=10)
        ax.axis("off")
        fig.colorbar(tmp, cax=fig.add_axes([0.92, 0.15, 0.04, 0.7]))
        plt.show()

In [ ]:
import tifffile
for i in range(num_comp):
    tilt_series = []
    for j in range(num_img):
        tilt_series.append(coeffs_reshape[j][:, :, i].astype(np.float32))
    tilt_series = np.asarray(tilt_series)
    tifffile.imsave("tilt_series_latent_%02d.tif"%(i+1), tilt_series)